[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pugapatricia/gestion-documentaria-para-pymes/blob/main/etiquetado/Etiquetado.ipynb)

[![Ver en GitHub](https://img.shields.io/badge/GitHub-Repo-black?logo=github)](https://github.com/pugapatricia/gestion-documentaria-para-pymes/tree/main/etiquetado)

In [1]:
!pip install -qq PyPDF2 python-docx openpyxl python-pptx xlrd transformers office365-rest-python-client msal requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 5.3 MB/s eta 0:00:00


In [2]:
import os
import io
import re
import json
from pathlib import Path
from PyPDF2 import PdfReader
import docx
import openpyxl
from pptx import Presentation
import xlrd
from transformers import pipeline
from office365.sharepoint.client_context import ClientContext
from office365.runtime.auth.user_credential import UserCredential
import os
import requests
import msal


# Configuración

In [3]:
CLIENT_ID = "e3f2393e-7348-47d1-9c64-8d8efe6a5e95"  # tu nuevo Client ID
AUTHORITY = "https://login.microsoftonline.com/consumers"
SCOPE = ["User.Read", "Files.ReadWrite"]

# Autenticación con Device Code Flow

In [12]:
app = msal.PublicClientApplication(CLIENT_ID, authority=AUTHORITY)

flow = app.initiate_device_flow(scopes=SCOPE)
if "user_code" not in flow:
    raise Exception("No se pudo iniciar el device flow. Revisa tu configuración en Azure.")

print(flow["message"])
result = app.acquire_token_by_device_flow(flow)

if "access_token" not in result:
    raise Exception(f"Error autenticación: {result.get('error_description')}")

access_token = result["access_token"]

# =========================
# Definir headers para Graph API
# =========================
headers = {"Authorization": f"Bearer {access_token}"}

# Verificar usuario autenticado
me = requests.get(
    "https://graph.microsoft.com/v1.0/me",
    headers=headers
).json()
print("✅ Usuario autenticado:", me.get("userPrincipalName"))


To sign in, use a web browser to open the page https://www.microsoft.com/link and enter the code V8QY6NB3 to authenticate.
✅ Usuario autenticado: marcomendieta08@gmail.com


In [13]:
me = requests.get(
    "https://graph.microsoft.com/v1.0/me",
    headers={"Authorization": f"Bearer {access_token}"}
).json()
print("✅ Usuario autenticado:", me.get("userPrincipalName"))

✅ Usuario autenticado: marcomendieta08@gmail.com


# Llamada a OneDrive

In [14]:
url = "https://graph.microsoft.com/v1.0/me/drive/root:/Etiquetados:/children"
resp = requests.get(url, headers={"Authorization": f"Bearer {access_token}"})
data = resp.json()

print("\nArchivos en la carpeta Etiquetados:")
for item in data.get("value", []):
    if "folder" not in item:  # 👉 esto asegura que sea archivo, no carpeta
        print("-", item.get("name"), "→", item.get("webUrl", "sin URL"))


Archivos en la carpeta Etiquetados:
- .DS_Store → https://onedrive.live.com?cid=9F68595D59F0B70B&id=9F68595D59F0B70B!s49f62d124226477da7f2a291252612d9
- 11_07_2019_modelo_orientativo_de_contrato_de_arrendamiento_de_vivienda.pdf → https://onedrive.live.com?cid=9F68595D59F0B70B&id=9F68595D59F0B70B!sc1bb0c8d90404e3f8ae92438b23c4747
- 2016-admitidos_Segundo ciclo- Cursos monográficos.xls → https://onedrive.live.com/personal/9f68595d59f0b70b/_layouts/15/doc.aspx?resid=78aae573-4f90-4067-84d2-5ad06797bcc6&cid=9f68595d59f0b70b
- 2023_05-Modelo_Documento_reserva_inmueble_en_alquiler_v.reducida.docx → https://onedrive.live.com/personal/9f68595d59f0b70b/_layouts/15/doc.aspx?resid=c776b69a-0375-4771-b022-1ba01443bd62&cid=9f68595d59f0b70b
- 660d1bfb7c43622a597a4000_Non-disclosure agreement nda template contract.pdf → https://onedrive.live.com?cid=9F68595D59F0B70B&id=9F68595D59F0B70B!s4a608a7d57454825ad5d41ffb84e3a6b
- Acuerdo_no_Divulgacion_Unilateral_UE.pdf → https://onedrive.live.com?cid=9F6859

# Lista de etiquetas personalizadas

In [15]:
etiquetas = [
    "Finanzas", "Contabilidad", "FacturasEmitidas", "FacturasRecibidas",
    "Nóminas", "Bancos", "RecursosHumanos", "Contratos", "CVsCandidatos",
    "Formación", "PolíticasInternas", "Legal", "Clientes", "Proveedores",
    "LicenciasPermisos", "Operaciones", "Proyectos", "Procesos", "Calidad"
]

ext_permitidas = {"pdf", "docx", "xlsx", "xls", "pptx", "txt", "csv"}


# 4. Clasificador Hugging Face Zero-Shot


In [16]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

def limpiar_texto(texto: str) -> str:
    if not texto:
        return ""
    texto = texto.lower()
    texto = re.sub(r"\s+", " ", texto)
    texto = re.sub(r"[^a-záéíóúñ0-9\s]", "", texto)  # elimina caracteres raros
    return texto.strip()

# =========================
# Función para clasificar un texto
# =========================
def clasificar_texto(texto: str, etiquetas: list, threshold: float = 0.5) -> list:
    """
    Clasifica un texto en base a las etiquetas definidas.
    Devuelve solo las etiquetas cuyo score >= threshold.
    """
    texto_limpio = limpiar_texto(texto)
    result = classifier(
        texto_limpio,
        candidate_labels=etiquetas,
        multi_label=True
    )
    # Filtrar etiquetas por score
    etiquetas_finales = [
        label for label, score in zip(result["labels"], result["scores"]) if score >= threshold
    ]
    return etiquetas_finales


Device set to use cuda:0


# 5. Lectores de documentos

In [17]:
def leer_archivo(nombre, contenido, limite=2000):
    ext = nombre.split(".")[-1].lower()
    texto = ""
    try:
        if ext == "pdf":
            reader = PdfReader(io.BytesIO(contenido))
            for page in reader.pages:
                if page.extract_text():
                    texto += page.extract_text() + "\n"
                    if len(texto) >= limite: break
        elif ext == "docx":
            doc = docx.Document(io.BytesIO(contenido))
            for p in doc.paragraphs:
                if p.text.strip():
                    texto += p.text + "\n"
                    if len(texto) >= limite: break
        elif ext == "xlsx":
            wb = openpyxl.load_workbook(io.BytesIO(contenido), data_only=True, read_only=True)
            for sheet in wb.worksheets:
                for row in sheet.iter_rows(values_only=True):
                    texto += " ".join([str(cell) for cell in row if cell]) + "\n"
                    if len(texto) >= limite: break
        elif ext == "xls":
            with open("temp.xls", "wb") as f: f.write(contenido)
            wb = xlrd.open_workbook("temp.xls")
            for sheet in wb.sheets():
                for row_idx in range(sheet.nrows):
                    row = sheet.row_values(row_idx)
                    texto += " ".join([str(cell) for cell in row if cell]) + "\n"
                    if len(texto) >= limite: break
        elif ext == "pptx":
            with open("temp.pptx", "wb") as f: f.write(contenido)
            prs = Presentation("temp.pptx")
            for slide in prs.slides:
                for shape in slide.shapes:
                    if hasattr(shape, "text") and shape.text.strip():
                        texto += shape.text + "\n"
                        if len(texto) >= limite: break
        elif ext in ["txt", "csv"]:
            texto = contenido.decode("utf-8", errors="ignore")[:limite]
    except Exception as e:
        print(f"⚠️ Error leyendo {nombre}: {e}")
        return ""
    return limpiar_texto(texto[:limite])

# 6. Función de etiquetado

In [11]:
def etiquetar_texto(texto):
    res = classifier(texto, candidate_labels=etiquetas, multi_label=True)
    return [label for label, score in zip(res["labels"], res["scores"]) if score > 0.3]

# 7. Leer archivos de OneDrive

In [18]:
url = "https://graph.microsoft.com/v1.0/me/drive/root:/Etiquetados:/children"
response = requests.get(url, headers=headers)
data = response.json()

resultados = {}
for item in data.get("value", []):
    nombre = item["name"]
    if not any(nombre.lower().endswith(ext) for ext in ext_permitidas):
        continue
    print(f"📂 Procesando {nombre}...")

    # Descargar archivo
    download_url = item["@microsoft.graph.downloadUrl"]
    file_bytes = requests.get(download_url).content

    # Extraer texto
    texto = leer_archivo(nombre, file_bytes)
    if texto:
        etiquetas_detectadas = etiquetar_texto(texto)
        resultados[nombre] = etiquetas_detectadas
        print(f"✅ {nombre} → {etiquetas_detectadas}")
    else:
        resultados[nombre] = []
        print(f"⚠️ No se pudo leer {nombre}")

📂 Procesando 11_07_2019_modelo_orientativo_de_contrato_de_arrendamiento_de_vivienda.pdf...
✅ 11_07_2019_modelo_orientativo_de_contrato_de_arrendamiento_de_vivienda.pdf → ['Legal', 'Contratos', 'Contabilidad', 'Formación', 'Calidad', 'Operaciones', 'Procesos', 'Proyectos']
📂 Procesando 2016-admitidos_Segundo ciclo- Cursos monográficos.xls...
✅ 2016-admitidos_Segundo ciclo- Cursos monográficos.xls → ['RecursosHumanos', 'Procesos', 'Proyectos', 'Calidad', 'Contabilidad', 'Contratos', 'FacturasRecibidas', 'Formación', 'FacturasEmitidas', 'CVsCandidatos', 'Operaciones', 'PolíticasInternas', 'Proveedores', 'Clientes', 'Legal', 'LicenciasPermisos']
📂 Procesando 2023_05-Modelo_Documento_reserva_inmueble_en_alquiler_v.reducida.docx...
✅ 2023_05-Modelo_Documento_reserva_inmueble_en_alquiler_v.reducida.docx → ['Calidad', 'Formación', 'Contratos', 'Procesos', 'Proyectos', 'Contabilidad', 'Operaciones', 'FacturasRecibidas']
📂 Procesando 660d1bfb7c43622a597a4000_Non-disclosure agreement nda template

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


✅ biblioteca2-csv.xls → ['Contratos', 'Proyectos', 'Contabilidad', 'Calidad', 'Formación', 'Procesos', 'Proveedores']
📂 Procesando contrato_alquiler_opcion_compra.pdf...
✅ contrato_alquiler_opcion_compra.pdf → ['Contratos', 'Contabilidad', 'Proyectos', 'Calidad', 'Procesos', 'Formación', 'Operaciones', 'Clientes', 'FacturasRecibidas']
📂 Procesando ContratoAlquiler_reducido.pdf...
✅ ContratoAlquiler_reducido.pdf → ['Contratos', 'Contabilidad', 'Proyectos', 'Procesos', 'Formación', 'Calidad', 'Operaciones', 'Finanzas', 'RecursosHumanos', 'FacturasRecibidas', 'Nóminas', 'Proveedores', 'CVsCandidatos', 'Legal', 'FacturasEmitidas', 'Bancos', 'PolíticasInternas', 'LicenciasPermisos']
📂 Procesando contrato-de-confidencialidad-freelancer-plantilla-gratis-word.docx...
✅ contrato-de-confidencialidad-freelancer-plantilla-gratis-word.docx → ['Contabilidad', 'Clientes', 'Formación', 'Finanzas', 'Legal', 'Contratos', 'Calidad', 'Proyectos', 'Operaciones', 'FacturasRecibidas', 'Procesos']
📂 Procesand

# 8. Guardar resultados

In [19]:
with open("etiquetas_onedrive.json", "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

print("📌 Resultados guardados en etiquetas_onedrive.json")

📌 Resultados guardados en etiquetas_onedrive.json


In [20]:
# ============================
# Configuración
# ============================
json_path = "etiquetas_onedrive.json"
headers = {"Authorization": f"Bearer {access_token}"}

# Carpeta objetivo en OneDrive
carpeta_objetivo = "Etiquetados"

# ============================
# Cargar JSON de etiquetas
# ============================
with open(json_path, "r", encoding="utf-8") as f:
    etiquetas_data = json.load(f)

# ============================
# Aplicar etiquetas en la descripción
# ============================
for archivo, etiquetas in etiquetas_data.items():
    if not etiquetas:
        continue

    # Buscar archivo en OneDrive
    url_file = f"https://graph.microsoft.com/v1.0/me/drive/root:/{carpeta_objetivo}/{archivo}"
    resp_file = requests.get(url_file, headers=headers)

    if resp_file.status_code != 200:
        print(f"⚠️ No se encontró {archivo} en OneDrive")
        continue

    file_id = resp_file.json()["id"]

    # Guardamos etiquetas en el campo "description"
    url_update = f"https://graph.microsoft.com/v1.0/me/drive/items/{file_id}"
    payload = {"description": ", ".join(etiquetas)}

    response_update = requests.patch(
        url_update,
        headers={**headers, "Content-Type": "application/json"},
        json=payload
    )

    if response_update.status_code in [200, 204]:
        print(f"✅ Etiquetas {etiquetas} aplicadas a {archivo} en la descripción")
    else:
        print(f"⚠️ Error al actualizar {archivo}: {response_update.text}")


✅ Etiquetas ['Legal', 'Contratos', 'Contabilidad', 'Formación', 'Calidad', 'Operaciones', 'Procesos', 'Proyectos'] aplicadas a 11_07_2019_modelo_orientativo_de_contrato_de_arrendamiento_de_vivienda.pdf en la descripción
✅ Etiquetas ['RecursosHumanos', 'Procesos', 'Proyectos', 'Calidad', 'Contabilidad', 'Contratos', 'FacturasRecibidas', 'Formación', 'FacturasEmitidas', 'CVsCandidatos', 'Operaciones', 'PolíticasInternas', 'Proveedores', 'Clientes', 'Legal', 'LicenciasPermisos'] aplicadas a 2016-admitidos_Segundo ciclo- Cursos monográficos.xls en la descripción
✅ Etiquetas ['Calidad', 'Formación', 'Contratos', 'Procesos', 'Proyectos', 'Contabilidad', 'Operaciones', 'FacturasRecibidas'] aplicadas a 2023_05-Modelo_Documento_reserva_inmueble_en_alquiler_v.reducida.docx en la descripción
✅ Etiquetas ['Contratos', 'Legal', 'Procesos', 'Contabilidad', 'Clientes', 'Calidad', 'RecursosHumanos', 'Operaciones', 'CVsCandidatos', 'Formación', 'LicenciasPermisos', 'Nóminas', 'Finanzas', 'FacturasRecib